In [1]:
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

CLIENT_SECRET_FILE = 'client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']

flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
credentials = flow.run_console()
youtube = build('youtube', 'v3', credentials=credentials)


#https://www.youtube.com/channel/UCC5hntNsVE7rQRYqqv_ON0g/


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=233696733563-bn230l7i4kmq8ovb31jf0v7af2e8bnht.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=3hUuISNrCCDOIAkMsh2QMdcvnseG4i&prompt=consent&access_type=offline
Enter the authorization code: 4/zQHMph1pmLz_9DNR2V3m-kniR_slWaNANsOrxBfObJ2LocvbYA4rjWc


In [2]:
import pandas as pd

def show_playlist():
    request = youtube.playlists().list(
            part="snippet,contentDetails",
            mine=True,
            maxResults=50,
        )
    response = request.execute()
    v=[{ 'id': item['id'], 'title': item['snippet']['title'] ,'count': item["contentDetails"]["itemCount"]} for item in response['items']]
    print(v)
    return v
    

def show_videolist_bypage(token):
    request = youtube.search().list(
        part="snippet",
        forMine=True,
        type="video",
        maxResults=50,
        order="date",
        pageToken=token
    )
    response = request.execute()
    #print(response)
    return response

def show_videolist():
    data =pd.DataFrame()
    count =0
    size=50
    max=1
    nextPageToken =None
    while(count*size <= max) :
        rs=show_videolist_bypage(nextPageToken)
        max=int(rs['pageInfo']['totalResults'])
        nextPageToken= rs['nextPageToken']
        val=[{'id':d['id']['videoId'], 'title' :d['snippet']['title'] }  for d in rs['items']] 
        print(len(val),max,nextPageToken)
        data=data.append(val)
        count+=1
    
    return data

def create_playlist(youtube, title):
    request = youtube.playlists().insert(
        part="snippet,status",
        body={
          "snippet": {
            "title": "ML & AI |" + title,
            "description": "ML & AI |" + title,
            "tags": [
              "ML,AI "          
            ],
            "defaultLanguage": "en"
          },
          "status": {
            "privacyStatus": "Unlisted"
          }
        }
    )
    response = request.execute()
    if(response is not None and 'id' in response) :
        print("playlist added successfully with id :{0}".format(response["id"]))
    else:
          print("failed to create playlist ")
          print(response)
        
def add_playlist_item(youtube,playlist_id, playlist_item_id):
    request = youtube.playlistItems().insert(
        part="snippet",
        body={
          "snippet": {
            "playlistId": playlist_id,
            "position": 0,
            "resourceId": {
              "kind": "youtube#video",
              "videoId": playlist_item_id
            }
          }
        }
    )
    response = request.execute()
    print(response)

def delete_playlist(playlist_id):
     request =  youtube.playlists().delete(id=playlist_id)
     response = request.execute()
     print(response)
        

In [10]:
##video listing
#rs= show_videolist()
#rs.to_csv('youtube_videlist.csv')

#add videolist item
#add_playlist_item(youtube,"PL9tQy71sPLeHoB-g1H08J1hR8ZsXWzz7Z","05yT5rR-XsM")

#show playlists
#rs= show_playlist()

#create playlist

#create_playlist(youtube,"")

#delete_playlist()



In [3]:
import os
import pandas as pd

def create_playlist_data(fullpath):
    filter_extension=".ts"
    data = [ { 'playlist' : '|'.join(path.split('\\')[6:]) , 'file_name':file,  "path" : path}  for path, subdirs, files in os.walk(fullpath) for file in files if file.endswith(filter_extension)]
    df= pd.DataFrame(data)
    df.to_csv('youtube_upload_status.csv')
    groups=df.groupby(['playlist'])
    return groups.groups.keys()

dt=create_playlist_data(r"C:\Users\Ashwini\Downloads\PGDM\Video")

#print(len(dt))

for g in dt:
    print(g)
    create_playlist(youtube,g)
    

Course 0(Pre-program Preparatory Content)|M-2(Data Structures in Python)


HttpError: <HttpError 403 when requesting https://www.googleapis.com/youtube/v3/playlists?part=snippet%2Cstatus&alt=json returned "The user has created too many playlists recently. Please try the request again later.">

In [84]:
show_playlist(youtube)

PLMCuzyPpc7o50NuMy7cFLSdcfKaiPBHkn Course 0(Pre-program Preparatory Content)|M-7(Advanced SQL)|S-3(Window Functions) 0
PLMCuzyPpc7o4uBWi0uIqcql4tgPNdehsk Course 0(Pre-program Preparatory Content)|M-7(Advanced SQL)|S-2(Updating Table) 0
PLMCuzyPpc7o7o1PBvbdv5I0n7DsxLAnHA Course 0(Pre-program Preparatory Content)|M-7(Advanced SQL)|S-1(Database design) 0
PLMCuzyPpc7o59BGMtTXYJR8TXUo_Qm4of Course 0(Pre-program Preparatory Content)|M-6(Data Analysis using SQL) 0
PLMCuzyPpc7o5NurJB2t3IhUFPm4bD9U-2 Course 0(Pre-program Preparatory Content)|M-5(Data Visualisation in Python) 0
PLMCuzyPpc7o66c657hhjpg-N3usDKHyp_ Course 0(Pre-program Preparatory Content)|M-3(Python for Data Science)|S-3(Cleaning Data) 0
PLMCuzyPpc7o5O92DakZUidSWOxgW8nkmv Course 0(Pre-program Preparatory Content)|M-3(Python for Data Science)|S-2(Panda) 0
PLMCuzyPpc7o422cgitVfL_uQcdM1__yaL Course 0(Pre-program Preparatory Content)|M-3(Python for Data Science)|S-1(NumPy) 0
PLMCuzyPpc7o7wq4Qp9LN6NKdbY452IIxk Course 0(Pre-program Prep

In [ ]:
from googleapiclient.http import MediaFileUpload


def upload_video(youtube,file_path):
    request = youtube.videos().insert(
        part="snippet,status",
        body={
          "snippet": {
            "categoryId" :27, #education
            "tags": "Machine Learning and AI,API call", 
            "description": "",
            "title": "Test video upload."
          },
          "status": {
            "privacyStatus": "Unlisted"
          }
        },
        
        
        media_body=MediaFileUpload(file_path)
    )
    response = request.execute()
    
    if response is not None and 'id' in response:
          print("Video id {0} was successfully uploaded.".format(response['id']))
    else:
        print("The upload failed with an unexpected response: "+  file_path)
        
    #print(response)
    #https://developers.google.com/youtube/v3/guides/uploading_a_video

In [ ]:
path=r"C:\Users\Ashwini\Downloads\PGDM\Video\Course 1\M-1 (Intro)\Graphical Models-278020.ts"

#upload_video(youtube,path)